In [34]:
import pandas as pd
import numpy as np
import pyproj
import plotly.graph_objects as go

In [2]:
#Se importan los datos de los pozos del proyecto Kalé
pozos_kale = pd.read_csv(r'datasets/Pozos_Kale_CTM12.txt', delimiter=';', decimal=',')
pozo_inv_kale = pozos_kale[pozos_kale['Tipo']=='Investigación']
x_pozo_inv_kale, y_pozo_inv_kale  = pozo_inv_kale['ESTE'].values[0], pozo_inv_kale['NORTE'].values[0]
h_pozo_inv_kale = 3.902 #km
h_pozo_inv_kale_m = h_pozo_inv_kale*1000 #m
r_ext = 2*h_pozo_inv_kale_m+20000 #m

In [3]:
sismos = pd.read_csv(r'datasets\reporte_LBG_2.csv')
sismos['FECHA - HORA UTC']=sismos['Fecha  (UTC)'].astype(str)+' '+sismos['Hora  (UTC)'].astype(str)
sismos.rename(columns = {'Latitud(°)':'LATITUD (°)', 
                                'Longitud(°)':'LONGITUD (°)',
                                'Profundidad(Km)':'PROF. (Km)',
                                'Magnitud':'MAGNITUD',
                                'Tipo Magnitud':'TIPO MAGNITUD',
                                'Rms(Seg)':'RMS (Seg)',
                                'Gap(°)':'GAP (°)',
                                'Error  Latitud(Km)':'ERROR LATITUD (Km)',
                                'Error  Longitud(Km)':'ERROR LONGITUD (Km)',
                                'Error  Profundidad(Km)':'ERROR PROFUNDIDAD (Km)'}, inplace = True)
sismos.drop(['Fecha  (UTC)','Hora  (UTC)'],axis=1,inplace=True)
proj = pyproj.Proj("+proj=tmerc +lat_0=4.0 +lon_0=-73.0 +k=0.9992 +x_0=5000000 +y_0=2000000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs")
dfp=proj.transform(xx=sismos['LONGITUD (°)'],yy=sismos['LATITUD (°)'])
sismos['ESTE']=dfp[0]
sismos['NORTE']=dfp[1]

In [28]:
#metricas a WGS84
wgs84 = pyproj.Transformer.from_crs("epsg:9377", "epsg:4326")
#WGS84 A Metricas
magnas = pyproj.Transformer.from_crs("epsg:4326", "epsg:9377")

In [71]:
sismos.columns

Index(['LATITUD (°)', 'LONGITUD (°)', 'PROF. (Km)', 'MAGNITUD',
       'TIPO MAGNITUD', 'RMS (Seg)', 'GAP (°)', 'ERROR LATITUD (Km)',
       'ERROR LONGITUD (Km)', 'ERROR PROFUNDIDAD (Km)', 'Modelo',
       'FECHA - HORA UTC', 'ESTE', 'NORTE'],
      dtype='object')

In [77]:
cey=wgs84.transform(sismos['NORTE']+sismos['ERROR LATITUD (Km)'],sismos['ESTE'])[0]
cex=wgs84.transform(sismos['NORTE'],sismos['ESTE']+sismos['ERROR LONGITUD (Km)'])[1]
sismos['ERROR LONGITUD (°)']=cex-sismos['LONGITUD (°)']
sismos['ERROR LATITUD (°)']=cey-sismos['LATITUD (°)']

In [79]:
sismos['ERROR LATITUD (°)']

0       0.153812
1       0.621559
2       0.727395
3       0.063330
4       0.196328
          ...   
4336    0.038869
4337    0.017271
4338    0.009601
4339    0.010877
4340    0.008316
Name: ERROR LATITUD (°), Length: 4341, dtype: float64

In [29]:
magnas.transform(sismos['LATITUD (°)'], sismos['LONGITUD (°)'])

(array([2327995.18599111, 2522023.36396967, 2409795.0703939 , ...,
        2354550.65599076, 2361302.03395695, 2356374.32793293]),
 array([4873677.53342547, 4895430.74403669, 4869806.69769316, ...,
        4911733.50207763, 4967426.83519574, 4911626.33599162]))

In [32]:
region=[x_pozo_inv_kale-52000,x_pozo_inv_kale+52000,
        y_pozo_inv_kale-52000,y_pozo_inv_kale+52000]

In [ ]:
lso=magnas.transform(sismos['LATITUD (°)'], sismos['LONGITUD (°)'])
lse=magnas.transform(sismos['LATITUD (°)'], sismos['LONGITUD (°)'])
lio=magnas.transform(sismos['LATITUD (°)'], sismos['LONGITUD (°)'])
lie=magnas.transform(sismos['LATITUD (°)'], sismos['LONGITUD (°)'])

In [33]:
def cylinder(r, h,x0,y0,a =0, nt=600, nv=50):
    """
    r=radio
    h=altura
    a=altura sobre eje x y y (z)
    nt=Calidad en un eje
    nv=Calidad e otro eje
    x=coordenada x
    y=coordenada y
    """
    theta = np.linspace(0, 2*np.pi, nt)
    v = np.linspace(a, a+h, nv )
    theta, v = np.meshgrid(theta, v)
    x = r*np.cos(theta)+x0
    y = r*np.sin(theta)+y0
    z = v
    return x, y, z

def boundary_circle(r, h,x0,y0, nt=600):
    """
    r - radio del borde
    h - altura sobre el eje xy (z)
    nt-calidad de un eje
    x0-coordenada x
    y0-coordenada y
    """
    theta = np.linspace(0, 2*np.pi, nt)
    x= r*np.cos(theta)+x0
    y = r*np.sin(theta)+y0
    z = h*np.ones(theta.shape)
    return x, y, z

In [ ]:
df_topo =pd.read_csv('datasets\kale_100.xyz',delimiter=' ',header=None,decimal='.')
#df_topo =df_topo[(df_topo[1]>ys)&(df_topo[1]<yn)&(df_topo[0]>xw)&(df_topo[0]<xe)] #Filtros previos
mesh_topo = (df_topo.pivot(index=1, columns=0,values=2))
z_topo,x_topo,y_topo=mesh_topo.values,mesh_topo.columns,mesh_topo.index

In [37]:
def volumen_semaforo(r1,a1,h1,x01,y01,opac,col_cyl,col_cir,name):
    x1, y1, z1 = cylinder(r1, h1,x01,y01, a=a1)
    cyl1 = go.Surface(x=x1, y=y1, z=z1,
                    colorscale = col_cyl,
                    showscale=False,
                    opacity=opac,
                    name=name)
    xb_low, yb_low, zb_low = boundary_circle(r1, a1,x01,y01)
    xb_up, yb_up, zb_up = boundary_circle(r1, a1+h1,x01,y01)

    bcircles1 =go.Scatter3d(x = xb_low.tolist()+[None]+xb_up.tolist(),
                            y = yb_low.tolist()+[None]+yb_up.tolist(),
                            z = zb_low.tolist()+[None]+zb_up.tolist(),
                            mode ='lines',
                            line = dict(color=col_cir, width=2),
                            opacity =0.55, showlegend=False,
                            name=name)
    return cyl1,bcircles1

In [36]:
#Asignamos las dimensiones y ubicacion del cilindro interno y externo respectivamente
r1 = 2*h_pozo_inv_kale_m  #Radio interno es dos veces la profundidad medida del pozo. De acuerdo con Resolución 40185 del 2020 del MME. Profundidad aproximada en pozo de investigación es 3902 m
a1 = 0 #Altura
h1 = -16000 #Profundidad del cilindro de 16 km
x01=float(x_pozo_inv_kale)
y01=float(y_pozo_inv_kale)

In [39]:
cyl1,bcircles1=volumen_semaforo(r1,a1,h1,x01,y01,0.5,['red','red'],'red','Suspensión')

In [44]:
r1=2*h_pozo_inv_kale_m

In [45]:
i=wgs84.transform(2371970,4905500)
f=wgs84.transform(2371970+2*h_pozo_inv_kale_m,4905500)

In [54]:
geo_inv=wgs84.transform(2371970,4905500)

In [56]:
geo_inv

(7.36551588073962, -73.85657811741982)

In [53]:
r1

7804.0

In [57]:
f[0]-i[0]

0.07061378808522001

In [58]:
geo_r1=wgs84.transform(2371970+r1,4905500)

In [59]:
geo_r1[0]-geo_inv[0]

0.07061378808522001

In [63]:
xe=x_pozo_inv_kale+50000
xw=x_pozo_inv_kale-50000
ys=y_pozo_inv_kale-50000
yn=y_pozo_inv_kale+50000


In [70]:
print(wgs84.transform(ys,xe)[1])
print(wgs84.transform(yn,xe)[0])
print(wgs84.transform(ys,xw)[0])
print(wgs84.transform(yn,xw)[1])

-73.40297793835953
7.8186098149102365
6.912061719633063
-74.31109712811764


In [42]:
r1=2*h_pozo_inv_kale_m

In [80]:
xe=x_pozo_inv_kale+52000
xw=x_pozo_inv_kale-52000
ys=y_pozo_inv_kale-52000
yn=y_pozo_inv_kale+52000

ge=wgs84.transform(ys,xe)[1]
gw=wgs84.transform(yn,xw)[1]
gn=wgs84.transform(yn,xe)[0]
gs=wgs84.transform(ys,xw)[0]

In [82]:
ge,gw,gn,gs

(-73.384852230196, -74.32929781241609, 7.836724863369543, 6.893917274978344)

In [ ]:
def var_un(x_pozo_inv_kale, y_pozo_inv_kale,h_pozo_inv_kale_m,sismos,name):    
    #Se importan los datos de los pozos del proyecto Kalé

    # x_pozo_inv_kale, y_pozo_inv_kale  = 4905500, 2371970
    # h_pozo_inv_kale_m = 3902 #m
    # r_ext = 2*h_pozo_inv_kale_m+20000 #m

    geo_inv=wgs84.transform(y_pozo_inv_kale ,x_pozo_inv_kale)

    kale= go.Scatter3d(
        x=np.array(geo_inv[1]),
        y=np.array(geo_inv[0]),
        z=np.array(300),
        mode='markers',
        marker_symbol='diamond',
        name="PPII Kalé",
        hovertemplate ="PPII Kalé - Investigación",
        marker=dict(
            size=4,
            color='black'
        )
    )

    r1=2*h_pozo_inv_kale_m

    geo_r1=wgs84.transform(y_pozo_inv_kale+r1,x_pozo_inv_kale)
    geo_r2=wgs84.transform(y_pozo_inv_kale+r_ext,x_pozo_inv_kale)
    geo_r3=wgs84.transform(y_pozo_inv_kale+50000,x_pozo_inv_kale)

    #Volumen de suspension
    cyl1,bcircles1=volumen_semaforo(geo_r1[0]-geo_inv[0] ,0,-16000,
                        geo_inv[1],geo_inv[0],0.5,
                        ['red','red'],'red','Suspensión')
    #Volumen de monitoreo
    cyl2,bcircles2=volumen_semaforo(geo_r2[0]-geo_inv[0],0,-16000,
                        geo_inv[1],geo_inv[0],0.7,
                        ['green','orange'],'green','Monitoreo')
    #Volumen externo
    cyl3,bcircles3=volumen_semaforo(geo_r3[0]-geo_inv[0],0,-32000,
                        geo_inv[1],geo_inv[0],0.3,
                        ['aqua','aqua'],'blue','Externo')





    sismos_local = sismos[(sismos['PROF. (Km)']<=16)&
                    ((((sismos['ESTE']-x_pozo_inv_kale)**2)+
                    ((sismos['NORTE']-y_pozo_inv_kale)**2))
                    **(1/2)<=r_ext)]

    #Drenajes
    # df_rivers=pd.read_csv('datasets\drenajes.csv')
    Mc=1.8

    m0 = sismos[(sismos['MAGNITUD']<Mc)]
    m1 = sismos[(sismos['MAGNITUD']>=Mc)&(sismos['MAGNITUD']<2)]
    m2 = sismos[(sismos['MAGNITUD']>=2)&(sismos['MAGNITUD']<3)]
    m3 = sismos[(sismos['MAGNITUD']>=3)&(sismos['MAGNITUD']<4)]
    m4 = sismos[(sismos['MAGNITUD']>=4)]
    mn=(m0,m1,m2,m3,m4)
    # Cálculo de parámetros para el semáforo

    nmsod_m0 = max_sismos_diarios(m0)   #Número máximo de sismos observados diarios
    nmsod_m1 = max_sismos_diarios(m1)   #Número máximo de sismos observados diarios
    nmsod_m2 = max_sismos_diarios(m2)   #Número máximo de sismos observados diarios
    nmsod_m3 = max_sismos_diarios(m3)   #Número máximo de sismos observados diarios
    nmsod_m4 = max_sismos_diarios(m4)   #Número máximo de sismos observados diarios
    nmsod_mn=(nmsod_m0,nmsod_m1,nmsod_m2,nmsod_m3,nmsod_m4)
    ND = (sismos['UTC'].max()-sismos['UTC'].min()).days  #Número de días calendario del catálogo

    npsed_m0 = m0.shape[0]/ND    #Número promedio de sismos esperados diarios #OJO!!!!!!!!!
    npsod_m1 = m1.shape[0]/ND   #Número promedio de sismos observados diarios
    npsod_m2 = m2.shape[0]/ND   #Número promedio de sismos observados diarios
    npsod_m3 = m3.shape[0]/ND   #Número promedio de sismos observados diarios
    npsod_m4 = m4.shape[0]/ND   #Número promedio de sismos observados diarios
    npseod_mn=(npsed_m0,npsod_m1,npsod_m2,npsod_m3,npsod_m4)

    if nmsod_m1 == npsod_m1:
            d_m1 = 1 - (1/ND)
    else:
        d_m1 = (nmsod_m1/npsod_m1)/2 #Tolerancia frecuencial m1    

    if nmsod_m2 == npsod_m2:
        d_m2 = 1 - (1/ND)
    else:
        d_m2 = (nmsod_m2/npsod_m2)/2 #Tolerancia frecuencial m2
        
    if nmsod_m3 == npsod_m3:
        d_m3 = 1 - (1/ND)
    else:
        d_m3 = (nmsod_m3/npsod_m3)/2 #Tolerancia frecuencial m3

    if nmsod_m4 == npsod_m4:
        d_m4 = 1 - (1/ND)
    else:
        d_m4 = (nmsod_m4/npsod_m4)/2 #Tolerancia frecuencial m4
    d_mn=(d_m1,d_m2,d_m3,d_m4)

    xe=x_pozo_inv_kale+52000
    xw=x_pozo_inv_kale-52000
    ys=y_pozo_inv_kale-52000
    yn=y_pozo_inv_kale+52000

    ge=wgs84.transform(ys,xe)[1]
    gw=wgs84.transform(yn,xw)[1]
    gn=wgs84.transform(yn,xe)[0]
    gs=wgs84.transform(ys,xw)[0]

    sismos_total = sismos[(sismos['PROF. (Km)']<=32)&
                    (sismos['ESTE']<=xe)&
                    (sismos['ESTE']>=xw)&
                    (sismos['NORTE']<=yn)&
                    (sismos['ESTE']>=ys)]
    return sismos_total,sismos_local,Mc,ND,mn,nmsod_mn,npseod_mn,d_mn,x_pozo_inv_kale,y_pozo_inv_kale,r1,r_ext,gw,ge,gs,gn